# Python LP Sudoku Solver
In this notebook, I develop a program for importing the New York Times' daily sudoku puzzle and solving it using liear optimization. <br>
I use Selenium web-driver to login and import the data, then I use scipy optimize/pyomo to formulate and solve the problem

In [31]:
# Data
import numpy as np
import pandas as pd

# Python modules: time and tkinter for accessing clipboard
from tkinter import Tk
import time
import os
import re
import itertools

# Webscraping
# Selenium
#basics
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

#wait tools
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# chaining actions
from selenium.webdriver.common.action_chains import ActionChains

# selecting items from a dropdown.
from selenium.webdriver.support.ui import Select

# Beautiful Soup
from bs4 import BeautifulSoup

## Webscraping
create a driver for logging new york times sudoku and storing numbers in an array

In [103]:
class NYTSudoku:
    """
    A webscraper that scrapes the daily sudoku from the New York Times
    """
    
    # Initialize with the webdriver and link.
    def __init__(self):
        """
        instantiate the path to the webdriver
        and the address of the portal
        """
        # Vars for web
        self.path = "selenium webdriver/chromedriver.exe"
        self.web_address = "https://www.nytimes.com/puzzles/sudoku/"
        self.difficulty = 'medium'
        
        # Vars for storing webscraped data.
        self.sudoku_init = []


    def open_webdriver(self):
        """
        Opens the webdriver to the correct web_address.
        """
        # Open the chrome driver
        self.driver = webdriver.Chrome(self.path)

        # Get the CoA website and print the website title
        self.driver.get(self.web_address+self.difficulty)
        

    def close_webdriver(self, delay=0.5):
        """
        Closes the webdriver after a given time delay
        """
        time.sleep(delay)
        self.driver.quit()
        print('[Info] driver closed :)')


    def retrieve(self):
        """
        Retrieve and reformat the sudoku.
        """
        # get the entire board
        cells = WebDriverWait(self.driver, 20).until(EC.presence_of_element_located((By.CLASS_NAME, 'su-board')))
        cells_soup = cells.get_attribute('innerHTML')

        # string parse to get only text entries
        cells_l = re.split('aria-label=|class=', cells_soup)
        cells_l = [c.strip().replace('"','') for c in cells_l]
        c_set = ['1','2','3','4','5','6','7','8','9','empty']
        extracted = [x for x in cells_l if x in set(c_set)]

        # clean
        extracted = [int(x.replace('empty','0')) for x in extracted]

        extracted_r = []
        for i in range(9):
            n = []
            for j in range(9):
                n.append(9*i+j)
            
            print(extracted[n[0]:n[8]+1])
            extracted_r.append(extracted[n[0]:n[8]+1])
        
        # return
        return extracted_r

## Testing class from above:

Instantiated some basics above. Test the access:

In [99]:
sudoku_scraper = NYTSudoku()

sudoku_scraper.open_webdriver()

C:\Users\Keagan Rankin\AppData\Local\Temp\ipykernel_16044\3971571370.py:26: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  self.driver = webdriver.Chrome(self.path)


In [100]:
retreived = sudoku_scraper.retrieve()

[0, 0, 0, 2, 0, 0, 1, 6, 4]
[0, 0, 0, 0, 0, 9, 0, 0, 0]
[0, 0, 0, 0, 1, 0, 3, 0, 2]
[0, 5, 0, 0, 0, 7, 9, 4, 8]
[0, 8, 0, 9, 0, 5, 0, 0, 6]
[0, 0, 3, 0, 0, 2, 0, 0, 1]
[0, 0, 0, 0, 7, 0, 0, 0, 0]
[3, 0, 0, 0, 0, 0, 0, 0, 0]
[5, 2, 0, 0, 0, 0, 4, 8, 0]


In [269]:
retreived

[[0, 0, 0, 2, 0, 0, 1, 6, 4],
 [0, 0, 0, 0, 0, 9, 0, 0, 0],
 [0, 0, 0, 0, 1, 0, 3, 0, 2],
 [0, 5, 0, 0, 0, 7, 9, 4, 8],
 [0, 8, 0, 9, 0, 5, 0, 0, 6],
 [0, 0, 3, 0, 0, 2, 0, 0, 1],
 [0, 0, 0, 0, 7, 0, 0, 0, 0],
 [3, 0, 0, 0, 0, 0, 0, 0, 0],
 [5, 2, 0, 0, 0, 0, 4, 8, 0]]

## Working on scrapper below:

Get each sudoku value under div data-cell="n" n from 1-80 with aria-level="number"

In [17]:
clues = WebDriverWait(sudoku_scraper.driver, 20).until(EC.presence_of_element_located((By.CLASS_NAME, 'su-board')))
clues_soup = clues.get_attribute('innerHTML')
print(clues_soup)

<div data-cell="0" aria-label="empty" class="su-cell selected" style="top: 0px; left: 0px; width: 54px; height: 54px;"><div class="su-cell__conflict" style="width: 10px; height: 10px; bottom: 5px; right: 5px;"></div></div><div data-cell="1" aria-label="empty" class="su-cell" style="top: 0px; left: 55px; width: 54px; height: 54px;"><div class="su-cell__conflict" style="width: 10px; height: 10px; bottom: 5px; right: 5px;"></div></div><div data-cell="2" aria-label="empty" class="su-cell" style="top: 0px; left: 110px; width: 54px; height: 54px;"><div class="su-cell__conflict" style="width: 10px; height: 10px; bottom: 5px; right: 5px;"></div></div><div data-cell="3" aria-label="2" class="su-cell prefilled" style="top: 0px; left: 169px; width: 54px; height: 54px;"><svg class="su-cell__value su-stretch" number="2" weight="bold" viewBox="0 0 200 200" fill="none" xmlns="http://www.w3.org/2000/svg"><path class="su-number" d="M134.815 128.985H95.7692C100.338 125.523 107.815 120.123 112.108 116.24

In [41]:
clues_l = re.split('aria-label=|class=', clues_soup)

#clues_l = clues_soup.split("aria-label")

clues_l = [c.strip().replace('"','') for c in clues_l]
c_set = ['1','2','3','4','5','6','7','8','9','empty']

extracted = [x for x in clues_l if x in set(c_set)]

extracted

['empty',
 'empty',
 'empty',
 '2',
 'empty',
 'empty',
 '1',
 '6',
 '4',
 'empty',
 'empty',
 'empty',
 'empty',
 'empty',
 '9',
 'empty',
 'empty',
 'empty',
 'empty',
 'empty',
 'empty',
 'empty',
 '1',
 'empty',
 '3',
 'empty',
 '2',
 'empty',
 '5',
 'empty',
 'empty',
 'empty',
 '7',
 '9',
 '4',
 '8',
 'empty',
 '8',
 'empty',
 '9',
 'empty',
 '5',
 'empty',
 'empty',
 '6',
 'empty',
 'empty',
 '3',
 'empty',
 'empty',
 '2',
 'empty',
 'empty',
 '1',
 'empty',
 'empty',
 'empty',
 'empty',
 '7',
 'empty',
 'empty',
 'empty',
 'empty',
 '3',
 'empty',
 'empty',
 'empty',
 'empty',
 'empty',
 'empty',
 'empty',
 'empty',
 '5',
 '2',
 'empty',
 'empty',
 'empty',
 'empty',
 '4',
 '8',
 'empty']

## Optimizer
working on feeding the sudoku into an optimizer, and building some classes for it. See mathematical formulation here: https://vanderbei.princeton.edu/tex/talks/INFORMS_19/Sudoku.pdf
<br>
There is actually no objective function. We are simply satisfying the constraints as any feasible solution will work. What are the constraints? k= the number, i=row, j=column
- let $x_{ijk}$ = 1 if there is a number in the sudoku cell, 0 if not.
- each cell must contain a number at the end: $\sum_{k}{x_{ijk}}=1$ for i 1->9 and j 1->9
- each row must contain each number k: $\sum_{j}{x_{ijk}}=1$ for i and k 1->9
- each column must contain each number k: $\sum_{i}{x_{ijk}}=1$ for j and k 1->9
- each nxn (3x3) block in the sudoku must contain each number k: $\sum_{i=3i_0-2}^{3i_0}\sum_{j=3j_0-2}^{3j_0}{x_{ijk}}=1$ for i-naught and j-naught 1->3 this is just indexing the square structures in our 2-D sudoku matrix
- We cannot change the cells that we know with known number $P_{i,j}$, so $x_{i,j P_{i,j} =1}$

<br><br> relevant documentation:
https://developers.google.com/optimization/pack/multiple_knapsack <br>
https://developers.google.com/optimization/lp/lp_example <br>
with pyomo another package https://medium.com/@dhanalakotamohan314/creating-sudoku-solver-with-python-and-pyomo-in-easy-steps-fe22ec916090 <br>
OR tools simpler solution using their all different constraint https://github.com/google/or-tools/blob/stable/examples/python/sudoku_sat.py

In [107]:
from scipy.optimize import linprog
#!pip install ortools #https://developers.google.com/optimization/introduction/python
from ortools.linear_solver import pywraplp

In [311]:
# create solver using OR tools linear solver
#solver = pywraplp.Solver.CreateSolver(solver_id="GLOP")
solver = pywraplp.Solver('Sudoku Solver', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
#if not solver:
#    return

retreived_arr = np.array(retreived)
grid_dim = 9
block_dim = 3

# Defining i,j,k variable
i_r = range(grid_dim)
j_r = range(grid_dim)
k_r = range(grid_dim)
i_0 = range(0,grid_dim,block_dim)
j_0 = range(0,grid_dim,block_dim)
cell_vals = range(1,10)

x = {}
for i in i_r:
    for j in j_r:
        for k in k_r:
            x[i,j,k] = solver.BoolVar(f'x{i}{j}{k}')


#x = [[[solver.BoolVar(f'x{i}{j}{k}') for k in k_r] for j in j_r] for i in i_r]
print("Number of variables =", solver.NumVariables())

Number of variables = 729


In [312]:
x

{(0, 0, 0): x000,
 (0, 0, 1): x001,
 (0, 0, 2): x002,
 (0, 0, 3): x003,
 (0, 0, 4): x004,
 (0, 0, 5): x005,
 (0, 0, 6): x006,
 (0, 0, 7): x007,
 (0, 0, 8): x008,
 (0, 1, 0): x010,
 (0, 1, 1): x011,
 (0, 1, 2): x012,
 (0, 1, 3): x013,
 (0, 1, 4): x014,
 (0, 1, 5): x015,
 (0, 1, 6): x016,
 (0, 1, 7): x017,
 (0, 1, 8): x018,
 (0, 2, 0): x020,
 (0, 2, 1): x021,
 (0, 2, 2): x022,
 (0, 2, 3): x023,
 (0, 2, 4): x024,
 (0, 2, 5): x025,
 (0, 2, 6): x026,
 (0, 2, 7): x027,
 (0, 2, 8): x028,
 (0, 3, 0): x030,
 (0, 3, 1): x031,
 (0, 3, 2): x032,
 (0, 3, 3): x033,
 (0, 3, 4): x034,
 (0, 3, 5): x035,
 (0, 3, 6): x036,
 (0, 3, 7): x037,
 (0, 3, 8): x038,
 (0, 4, 0): x040,
 (0, 4, 1): x041,
 (0, 4, 2): x042,
 (0, 4, 3): x043,
 (0, 4, 4): x044,
 (0, 4, 5): x045,
 (0, 4, 6): x046,
 (0, 4, 7): x047,
 (0, 4, 8): x048,
 (0, 5, 0): x050,
 (0, 5, 1): x051,
 (0, 5, 2): x052,
 (0, 5, 3): x053,
 (0, 5, 4): x054,
 (0, 5, 5): x055,
 (0, 5, 6): x056,
 (0, 5, 7): x057,
 (0, 5, 8): x058,
 (0, 6, 0): x060,
 (0, 6, 1)

attempt adding constraints

In [313]:
# constraint 1 -> over i and j, the sum/value in each cell must be 1
for i in i_r:
    for j in j_r: 
        solver.Add(solver.Sum(x[i,j,k] for k in k_r) == 1)

In [314]:
# constraint 2 -> over i and k, the sum over j must be 1
for i in i_r:
    for k in k_r: 
        solver.Add(solver.Sum(x[i,j,k] for j in j_r) == 1)

In [315]:
# constraint 3 -> over i and k, the sum over j must be 1
for j in j_r:
    for k in k_r: 
        solver.Add(solver.Sum(x[i,j,k] for i in i_r) == 1)

In [316]:
# constraint 4: block constraint
for k in k_r:
    for p in i_0:
        for q in j_0: 
            #solver.Add(solver.Sum(x[i,j,k] for i in range(p,p+3) for j in range(q, q+3)) == 1)
            solver.Add(solver.Sum(x[p+i,j,k] for j in range(q,(q+block_dim)) for i in range(block_dim)) == 1)

In [317]:
# constraint 5 known cells must be assigned
# retreived in form r[i][j]
for i in i_r:
    for j in j_r:
        if retreived_arr[i,j] != 0:
            #print(retreived[i][j])
            solver.Add(x[i,j,retreived_arr[i,j]-1] == 1)

Run optimizer with input

In [318]:
print("Number of constraints =", solver.NumConstraints())
solver.Constraint()

# init objective function
solver.Minimize(0)

Number of constraints = 350


In [319]:
status = solver.Solve()

Display solution

In [320]:
results = np.zeros((9,9)).astype(np.int32)
results

array([[0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [321]:
status_d = {0:'OPTIMAL', 1:'FEASIBLE', 2:'INFEASIBLE', 3:'UNBOUNDED', 
          4:'ABNORMAL', 5:'MODEL_INVALID', 6:'NOT_SOLVED'}

if status == pywraplp.Solver.OPTIMAL:
    for i in i_r:
        for j in j_r:
            results[i,j] = sum((k + 1) * int(x[i, j, k].solution_value()) for k in k_r)

else:
    raise Exception('Unfeasible Sudoku: {}'.format(status_d[status]))

    
    
    
    
    print("Solution:")
    print("Objective value =", solver.Objective().Value())
    print("x =", x[i,j,k].solution_value())

In [322]:
results

array([[7, 9, 8, 2, 5, 3, 1, 6, 4],
       [2, 3, 1, 4, 6, 9, 8, 7, 5],
       [4, 6, 5, 7, 1, 8, 3, 9, 2],
       [6, 5, 2, 1, 3, 7, 9, 4, 8],
       [1, 8, 7, 9, 4, 5, 2, 3, 6],
       [9, 4, 3, 6, 8, 2, 7, 5, 1],
       [8, 1, 9, 5, 7, 4, 6, 2, 3],
       [3, 7, 4, 8, 2, 6, 5, 1, 9],
       [5, 2, 6, 3, 9, 1, 4, 8, 7]])

In [323]:
retreived

[[0, 0, 0, 2, 0, 0, 1, 6, 4],
 [0, 0, 0, 0, 0, 9, 0, 0, 0],
 [0, 0, 0, 0, 1, 0, 3, 0, 2],
 [0, 5, 0, 0, 0, 7, 9, 4, 8],
 [0, 8, 0, 9, 0, 5, 0, 0, 6],
 [0, 0, 3, 0, 0, 2, 0, 0, 1],
 [0, 0, 0, 0, 7, 0, 0, 0, 0],
 [3, 0, 0, 0, 0, 0, 0, 0, 0],
 [5, 2, 0, 0, 0, 0, 4, 8, 0]]

It works!<br>
now build it into a function and clean it up

In [349]:
def sudoku_solve(sudoku: np.ndarray, grid_dim = 9, block_dim = 3):
    """
    pass sudoku grid as (i,i) np array,
    optimize using mix int-lin programming,
    returns solved grid or exception if infeasible.
    """
    # -- Setup --
    # create solver using OR tools linear solver
    solver = pywraplp.Solver('Sudoku Solver', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
    if not solver:
        print('Solver error, exiting.')
        return

    # Defining i,j,k variable
    i_r = range(grid_dim)
    j_r = range(grid_dim)
    k_r = range(grid_dim)
    i_0 = range(0,grid_dim,block_dim)
    j_0 = range(0,grid_dim,block_dim)

    # Initialize the variables for the optimization.
    x = {}
    for i in i_r:
        for j in j_r:
            for k in k_r:
                x[i,j,k] = solver.BoolVar(f'x{i}{j}{k}')


    # -- Constraints --
    # Constraint 1: each cell must contain a number 1 through 9
    for i in i_r:
        for j in j_r: 
            solver.Add(solver.Sum(x[i,j,k] for k in k_r) == 1)

    # Constraint 2: each row must contain a number 1 through 9
    for i in i_r:
        for k in k_r: 
            solver.Add(solver.Sum(x[i,j,k] for j in j_r) == 1)

    # Constraint 3: each column must contain a number 1 through 9
    for j in j_r:
        for k in k_r: 
            solver.Add(solver.Sum(x[i,j,k] for i in i_r) == 1)

    # Constraint 4: each nxn block must contain a number 1 through 9
    for k in k_r:
        for p in i_0:
            for q in j_0:
                solver.Add(solver.Sum(x[p+i,j,k] for j in range(q,(q+block_dim)) for i in range(block_dim)) == 1)

    # Constraint 5/Known number init: if known numbers i,j,k must be = 1
    for i in i_r:
        for j in j_r:
            if sudoku[i,j] != 0:
                solver.Add(x[i,j,sudoku[i,j]-1] == 1)



    # -- Solve and Return --
    # init objective function
    solver.Minimize(0)
    status = solver.Solve()

    results = np.zeros((9,9)).astype(np.int32)
    status_d = {0:'OPTIMAL', 1:'FEASIBLE', 2:'INFEASIBLE', 3:'UNBOUNDED', 
          4:'ABNORMAL', 5:'MODEL_INVALID', 6:'NOT_SOLVED'}

    # store results or except
    if status == pywraplp.Solver.OPTIMAL:
        for i in i_r:
            for j in j_r:
                results[i,j] = sum((k + 1) * int(x[i, j, k].solution_value()) for k in k_r)
    else:
        raise Exception('Bad Sudoku: {}'.format(status_d[status]))

    return results
    

In [350]:
sudoku_solve(retreived_arr)

array([[7, 9, 8, 2, 5, 3, 1, 6, 4],
       [2, 3, 1, 4, 6, 9, 8, 7, 5],
       [4, 6, 5, 7, 1, 8, 3, 9, 2],
       [6, 5, 2, 1, 3, 7, 9, 4, 8],
       [1, 8, 7, 9, 4, 5, 2, 3, 6],
       [9, 4, 3, 6, 8, 2, 7, 5, 1],
       [8, 1, 9, 5, 7, 4, 6, 2, 3],
       [3, 7, 4, 8, 2, 6, 5, 1, 9],
       [5, 2, 6, 3, 9, 1, 4, 8, 7]])

In [341]:
retreived_arr

array([[0, 0, 0, 2, 0, 0, 1, 6, 4],
       [0, 0, 0, 0, 0, 9, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 3, 0, 2],
       [0, 5, 0, 0, 0, 7, 9, 4, 8],
       [0, 8, 0, 9, 0, 5, 0, 0, 6],
       [0, 0, 3, 0, 0, 2, 0, 0, 1],
       [0, 0, 0, 0, 7, 0, 0, 0, 0],
       [3, 0, 0, 0, 0, 0, 0, 0, 0],
       [5, 2, 0, 0, 0, 0, 4, 8, 0]])

## Misc
misc stuff

In [87]:
def read_file_sudoku(file_sudoku):
    """
    read_file_sudoku(arr) -> int, [list, list]
        dim -> sudoku dimension
        arr -> sudoku 2D list
    """
    with open(file_sudoku,'r') as f:
        dim = int(f.readline())
        arr = [[] for k in range(dim)]
        for k in range(dim):
            arr[k] = list(map(int,f.readline().split()))
    return dim, arr

In [88]:
read_file_sudoku('sudoku_9x9.txt')

(9,
 [[0, 0, 9, 0, 0, 6, 3, 0, 0],
  [0, 0, 1, 0, 8, 0, 4, 0, 0],
  [3, 8, 0, 4, 9, 0, 6, 0, 0],
  [0, 1, 2, 9, 0, 4, 8, 6, 0],
  [8, 0, 3, 0, 1, 2, 0, 9, 0],
  [9, 4, 7, 0, 0, 0, 0, 0, 0],
  [0, 0, 6, 2, 0, 9, 7, 5, 8],
  [0, 9, 0, 5, 7, 0, 1, 0, 6],
  [7, 0, 0, 8, 0, 0, 9, 2, 3]])

I initialy coded everything from ORTools documentation, but used this to check my answer as I was not initilizing x correctly (I was surprisingly accurate on everything else despite never using OR tools before lol)
https://www.kaggle.com/code/pintowar/modeling-a-sudoku-solver-with-or-tools

In [356]:
!pip install numpy